In [1]:
DEVELOPMENT_MODE = False
try:
    import google.colab

    IN_COLAB = True
    print("Running as a Colab notebook")
    %pip install git+https://github.com/jbloomAus/SAELens

except:
    IN_COLAB = False
    print("Running as a Jupyter notebook - intended for development only!")
    from IPython import get_ipython

    ipython = get_ipython()
    # Code to automatically update the HookedTransformer code as its edited without restarting the kernel
    ipython.magic("load_ext autoreload")
    ipython.magic("autoreload 2")

Running as a Jupyter notebook - intended for development only!


/tmp/ipykernel_2458464/740336833.py:16: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("load_ext autoreload")
/tmp/ipykernel_2458464/740336833.py:17: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("autoreload 2")


In [2]:
import torch
if torch.cuda.is_available():
    device = "cuda:1"
# elif torch.backends.mps.is_available():
#     device = "mps"
else:
    device = "cpu"
torch.set_grad_enabled(False)

In [3]:
from sae_lens import HookedSAETransformer

model: HookedSAETransformer = HookedSAETransformer.from_pretrained("gpt2-small").to(
    device
)

/local/storage1/zhu.3944/topk/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Loaded pretrained model gpt2-small into HookedTransformer
Moving model to device:  cuda


In [4]:
import tqdm
from sae_lens import SAE

hook_name_to_sae = []
for layer in tqdm.tqdm(range(12)):
    sae, cfg_dict, _ = SAE.from_pretrained(
        "gpt2-small-res-jb",
        f"blocks.{layer}.hook_resid_pre",
        device=device,
    )
    hook_name_to_sae.append(sae)

  0%|          | 0/12 [00:00<?, ?it/s]/local/storage1/zhu.3944/topk/.venv/lib/python3.12/site-packages/sae_lens/sae.py:145: UserWarning: 
This SAE has non-empty model_from_pretrained_kwargs. 
For optimal performance, load the model like so:
model = HookedSAETransformer.from_pretrained_no_processing(..., **cfg.model_from_pretrained_kwargs)
  warnings.warn(
100%|██████████| 12/12 [00:02<00:00,  4.42it/s]


In [5]:
from datasets import load_dataset

# Load the Skylion007/openwebtext dataset
dataset = load_dataset('Skylion007/openwebtext')

# Print the first example from the dataset
print(dataset['train'][0])

/local/storage1/zhu.3944/topk/.venv/lib/python3.12/site-packages/datasets/load.py:1454: FutureWarning: The repository for Skylion007/openwebtext contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/Skylion007/openwebtext
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


{'text': 'Port-au-Prince, Haiti (CNN) -- Earthquake victims, writhing in pain and grasping at life, watched doctors and nurses walk away from a field hospital Friday night after a Belgian medical team evacuated the area, saying it was concerned about security.\n\nThe decision left CNN Chief Medical Correspondent Sanjay Gupta as the only doctor at the hospital to get the patients through the night.\n\nCNN initially reported, based on conversations with some of the doctors, that the United Nations ordered the Belgian First Aid and Support Team to evacuate. However, Belgian Chief Coordinator Geert Gijs, a doctor who was at the hospital with 60 Belgian medical personnel, said it was his decision to pull the team out for the night. Gijs said he requested U.N. security personnel to staff the hospital overnight, but was told that peacekeepers would only be able to evacuate the team.\n\nHe said it was a "tough decision" but that he accepted the U.N. offer to evacuate after a Canadian medical t

In [6]:
layer_activation_file = []
for layer in range(12):
    layer_activation_file.append(open(f"small{layer}.npy", "wb"))

In [7]:
import numpy as np
length = int(0.001 * len(dataset['train']))
for idx in tqdm.tqdm(range(length)):
#for idx in tqdm.tqdm(range(10)):
    example = dataset['train'][idx]
    tokens = model.to_tokens([example['text']], prepend_bos=True)
    _, cache = model.run_with_cache_with_saes(tokens, saes=hook_name_to_sae)
    for layer in range(12):
        sae_acts = cache[f"blocks.{layer}.hook_resid_pre.hook_sae_acts_post"]
        live_feature = (sae_acts > 1e-3 )[0]
        seq_sim = live_feature.sum(0) / live_feature.shape[0]
        np.save(layer_activation_file[layer], seq_sim.cpu().detach().numpy())

  0%|          | 0/8013 [00:00<?, ?it/s]

  0%|          | 1/8013 [00:00<1:03:00,  2.12it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 48.00 MiB. GPU 0 has a total capacity of 23.67 GiB of which 12.69 MiB is free. Process 2455667 has 14.53 GiB memory in use. Including non-PyTorch memory, this process has 9.11 GiB memory in use. Of the allocated memory 8.83 GiB is allocated by PyTorch, and 45.76 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [8]:
length = int(0.001 * len(dataset['train']))
acts = []
for idx in tqdm.tqdm(range(length)):
    acts.append(np.load(f"activation0001_layer2.npy"))

acts = np.array(acts)
acts.max()

100%|██████████| 8013/8013 [00:01<00:00, 7739.43it/s]


0.8359375